Reference:
https://docs.dgl.ai/en/0.8.x/tutorials/blitz/index.html　


Contents:
- Node classfication
- Edge classfication
- Graph classification 
- Implement model with more dimentions
- Comparing GNN and Treditional Neural Network(SAGEConv)
- Sending the message within graph: Heterogeneous Graph



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# this notebook use gpu training. check install version: https://www.dgl.ai/pages/start.html
!pip install dgl-cu110 dglgo -f https://data.dgl.ai/wheels/repo.html &>/dev/null

# load library

In [5]:
import os
import pandas as pd
import numpy as np
import scipy.sparse as sp
import itertools
from tqdm import tqdm
import dgl
# DGL is an easy-to-use, high performance and scalable Python package for deep learning on graphs. DGL is framework agnostic, meaning if a deep graph model is a component of an end-to-end application, the rest of the logics can be implemented in any major frameworks, such as PyTorch, Apache MXNet or TensorFlow.
from dgl.data import DGLDataset
import dgl.nn as dglnn
import networkx as nx
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

ModuleNotFoundError: ignored

# load data(for node/link classification)

Cora dataset: 

In this case, we will use a citation network of papers called cora dataset.

- Data for papers and the papers cited in each paper.
- Each paper falls into 7 categories ('Neural_Networks', 'Reinforcement_Learning', 'Probabilistic_Methods', etc.)
- Each paper also has a binary feature of which specific technical words (1,433 words) are included in the paper

The Cora dataset consists of 2,708 scientific publications classified among 7 classes. The citation network consists of 5,429 links.   
Each publication is described by a 0/1 binary label indicating the absence/presence of the corresponding word from the dictionary, which consists of 1,433 unique words.  

![cora](https://production-media.paperswithcode.com/datasets/Cora-0000000700-ce1c5ec7_LD7pZnT.jpg)

In [7]:
# read data and preprocessing

content_path = "/content/drive/MyDrive/Job Tasks/Graph convolution netwoks Node classification/cora/cora.content"
cite_path = "/content/drive/MyDrive/Job Tasks/Graph convolution netwoks Node classification/cora/cora.cites"

with open(content_path, "r") as fp:
    contents = fp.readlines()
with open(cite_path, "r") as fp:
    cites = fp.readlines()

contents = np.array([np.array(l.strip().split("\t")) for l in contents])
paper_list, feat_list, label_list = np.split(contents, [1,-1], axis=1)
paper_list, label_list = np.squeeze(paper_list), np.squeeze(label_list)
# Paper -> Index dict
paper_dict = dict([(key, val) for val, key in enumerate(paper_list)])
# Label -> Index dict
labels = list(set(label_list))
label_dict = dict([(key, val) for val, key in enumerate(labels)])
# Edge_index
cites = [i.strip().split("\t") for i in cites]
cites = np.array([[paper_dict[i[0]], paper_dict[i[1]]] for i in cites], 
                 np.int64).T   # (2, edge)
cites = np.concatenate((cites, cites[::-1, :]), axis=1)  # (2, 2*edge) or (2, E)
# Degree
_, degree_list = np.unique(cites[0,:], return_counts=True)

In [ ]:
# Input
node_num = len(paper_list)
feat_dim = feat_list.shape[1]
stat_dim = 32
num_class = len(labels)
T = 2
feat_Matrix = torch.Tensor(feat_list.astype(np.float32))
X_Node, X_Quote = np.split(cites, 2, axis=0)
X_Node, X_Quote = torch.from_numpy(np.squeeze(X_Node)), \
                 torch.from_numpy(np.squeeze(X_Quote))
label_list = np.array([label_dict[i] for i in label_list])
label_list = torch.from_numpy(label_list)

In [ ]:
print('Node:', X_Node) # paper id
print('Quote:', X_Quote) # quote paper id
print('label:', label_list) # label (paper genre)
print('feat:', feat_Matrix) # feat

Node: tensor([ 163,  163,  163,  ..., 2258, 1887, 1686])
Quote: tensor([ 402,  659, 1696,  ..., 1887, 1902,  837])
label: tensor([3, 2, 6,  ..., 4, 0, 3])
feat: tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


Creating a graph object
- A graph can be created by dgl.graph((source node, destination node), number of nodes)
- This graph is a directed graph, but if you want to treat it as an undirected graph, you can include g = dgl.to_bidirected(g) to reflect "edges facing the opposite direction" as well (not used in this example).

In [ ]:
g = dgl.graph((X_Node, X_Quote), num_nodes=node_num)
g.ndata['feat'] = feat_Matrix
g.ndata['label'] = label_list

In [ ]:
# appendix: dgl design seems to be based on NetworkX and is compatible.
# If you want to do visualization and have some affinity to networkx, dgl is better than pytorch geometric.

'''
# It takes time so cut
nx_graph = dgl.to_networkx(graph)
nx.draw_networkx(nx_graph)
plt.show()
'''

'\n# It takes time so cut\nnx_graph = dgl.to_networkx(graph)\nnx.draw_networkx(nx_graph)\nplt.show()\n'

# example1: Node prediction using GCN
https://docs.dgl.ai/tutorials/blitz/1_introduction.html

In [ ]:
# split train/valid/test

n_nodes = node_num
n_train = int(n_nodes * 0.6)
n_val = int(n_nodes * 0.2)
train_mask = torch.zeros(n_nodes, dtype=torch.bool)
val_mask = torch.zeros(n_nodes, dtype=torch.bool)
test_mask = torch.zeros(n_nodes, dtype=torch.bool)
train_mask[:n_train] = True
val_mask[n_train:n_train + n_val] = True
test_mask[n_train + n_val:] = True
g.ndata['train_mask'] = train_mask
g.ndata['val_mask'] = val_mask
g.ndata['test_mask'] = test_mask

In [ ]:
# attach GPU
g = g.to('cuda')

In [ ]:
print('Graph summary')
print(g)
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Graph summary
Graph(num_nodes=2708, num_edges=10858,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})
Node features
{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'), 'label': tensor([3, 2, 6,  ..., 4, 0, 3], device='cuda:0'), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False], device='cuda:0'), 'val_mask': tensor([False, False, False,  ..., False, False, False], device='cuda:0'), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True], device='cuda:0')}
Edge features
{}


## Model
- In the GraphConv layer, convolution is achieved by linearly combining the features of its own node with those of its neighboring nodes (convolution is performed from the feature dimension of the node to an arbitrary dimension).
- The forward method takes a graph object and feature values as arguments and outputs (node * dimension after convolution).
- For more details, read the official document: https://docs.dgl.ai/generated/dgl.nn.pytorch.conv.GraphConv.html#dgl.nn.pytorch.conv.GraphConv

In [ ]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [ ]:
def train(g, model, epoch, lr):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    
    
    for e in range(epoch):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)
        
        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 50 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

In [ ]:
# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, len(g.ndata['label'].unique())).to('cuda')
train(g, model, epoch = 500, lr = 0.001)

In epoch 0, loss: 1.933, val acc: 0.207 (best 0.207), test acc: 0.190 (best 0.190)
In epoch 50, loss: 1.206, val acc: 0.754 (best 0.754), test acc: 0.694 (best 0.691)
In epoch 100, loss: 0.694, val acc: 0.843 (best 0.843), test acc: 0.761 (best 0.761)
In epoch 150, loss: 0.441, val acc: 0.845 (best 0.850), test acc: 0.790 (best 0.772)
In epoch 200, loss: 0.319, val acc: 0.847 (best 0.850), test acc: 0.797 (best 0.772)
In epoch 250, loss: 0.249, val acc: 0.843 (best 0.850), test acc: 0.792 (best 0.772)
In epoch 300, loss: 0.205, val acc: 0.845 (best 0.850), test acc: 0.790 (best 0.772)
In epoch 350, loss: 0.173, val acc: 0.847 (best 0.850), test acc: 0.792 (best 0.772)
In epoch 400, loss: 0.150, val acc: 0.837 (best 0.850), test acc: 0.792 (best 0.772)
In epoch 450, loss: 0.131, val acc: 0.837 (best 0.850), test acc: 0.796 (best 0.772)


Here is the Accuracy of GCN on Node Prediction: 81%

I tried to see how the accuracy compares to lightgbm with only 1432 dimensional features.

In [ ]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score

In [ ]:
lgb_df = pd.DataFrame(feat_list.astype(np.float32))
lgb_df['target'] = label_list
lgb_df.head(3)

0    1    2    3    4    5    6    7    8    9  ...  1424  1425  1426  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   1.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   1.0   0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   

   1427  1428  1429  1430  1431  1432  target  
0   0.0   0.0   0.0   0.0   0.0   0.0       3  
1   0.0   0.0   0.0   0.0   0.0   0.0       2  
2   0.0   0.0   0.0   0.0   0.0   0.0       6  

[3 rows x 1434 columns]

In [ ]:
lgb_train_df = lgb_df[:n_train]
lgb_val_df = lgb_df[n_train:n_train + n_val]
lgb_test_df = lgb_df[n_train + n_val:]
print(lgb_train_df.shape)
print(lgb_val_df.shape)
print(lgb_test_df.shape)

(1624, 1434)
(541, 1434)
(543, 1434)


In [ ]:
target_col = 'target'
train_y = lgb_train_df[target_col]
train_x = lgb_train_df.drop(target_col, axis=1)

val_y = lgb_val_df[target_col]
val_x = lgb_val_df.drop(target_col, axis=1)

test_y = lgb_test_df[target_col]
test_x = lgb_test_df.drop(target_col, axis=1)

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

In [ ]:
params = {
    "objective": "multiclass",
    "boosting" : "gbdt",
    "num_class" : len(labels),
    "learning_rate": 0.01,       
    "metric": "multi_logloss", 
    "seed": 42,
}

In [ ]:
model = lgb.train(params, trains, valid_sets=valids, num_boost_round=1000, early_stopping_rounds=100, verbose_eval=100)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004914 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 784
[LightGBM] [Info] Number of data points in the train set: 1624, number of used features: 392
[LightGBM] [Info] Start training from score -2.166901
[LightGBM] [Info] Start training from score -1.916184
[LightGBM] [Info] Start training from score -2.757919
[LightGBM] [Info] Start training from score -1.178039
[LightGBM] [Info] Start training from score -1.981001
[LightGBM] [Info] Start training from score -1.985476
[LightGBM] [Info] Start training from score -2.388701
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's multi_logloss: 1.05589
[200]	valid_0's multi_logloss: 0.888613
[300]	valid_0's multi_logloss: 0.844305
[400]	valid_0's multi_logloss: 0.844727
Early stopping, best iteration is:
[329]	valid_0's multi_logloss: 0.83988


In [ ]:
val_pred = model.predict(val_x).argmax(1)
print('validation data accuracy:', accuracy_score(val_y, val_pred))
test_pred = model.predict(test_x).argmax(1)
print('test data accuracy:', accuracy_score(test_y, test_pred))

validation data accuracy: 0.7264325323475046
test data accuracy: 0.7034990791896869


Accuracy:

The ACCURACY in GNN was val: 0.845 test: 0.820, so it's definitely more accurate than lgb, which simply used node-only features!


- In this example, there was a "feature" in the node, but in a simple graph, there may be many cases where there is no feature in the node.
- In such cases, the node is regarded as an n-dimensional embeddable vector and is used as a feature.

In [ ]:
embedding = nn.Embedding(n_nodes, 128)  # nodes with embedding dim equal to 128
g.ndata['feat'] = embedding.weight.to('cuda')

In [ ]:
model = GCN(g.ndata['feat'].shape[1], 16, len(g.ndata['label'].unique())).to('cuda')
train(g, model, epoch = 500, lr = 0.001)

In epoch 0, loss: 1.989, val acc: 0.128 (best 0.128), test acc: 0.158 (best 0.158)
In epoch 50, loss: 1.709, val acc: 0.270 (best 0.270), test acc: 0.284 (best 0.284)
In epoch 100, loss: 1.511, val acc: 0.346 (best 0.346), test acc: 0.341 (best 0.341)
In epoch 150, loss: 1.331, val acc: 0.375 (best 0.375), test acc: 0.365 (best 0.366)
In epoch 200, loss: 1.162, val acc: 0.410 (best 0.410), test acc: 0.383 (best 0.381)
In epoch 250, loss: 1.011, val acc: 0.427 (best 0.427), test acc: 0.401 (best 0.401)
In epoch 300, loss: 0.887, val acc: 0.449 (best 0.449), test acc: 0.394 (best 0.394)
In epoch 350, loss: 0.787, val acc: 0.447 (best 0.453), test acc: 0.424 (best 0.400)
In epoch 400, loss: 0.703, val acc: 0.449 (best 0.453), test acc: 0.448 (best 0.400)
In epoch 450, loss: 0.632, val acc: 0.449 (best 0.455), test acc: 0.449 (best 0.442)


# example2: Link prediction using GCN
https://docs.dgl.ai/tutorials/blitz/4_link_predict.html


- In link prediction, "Given the features of nodeA and nodeB, is there a link between them?" The link prediction models "When there are features of nodeA and nodeB, is there a link between them?
- In the previous example, the node features corresponded to the node labels, but this time, the inner product of the features of nodeA and nodeB (corresponding to a link) corresponds to the label, and the binary classification of whether or not there is a link between the nodes is solved.
- When predicting links, the recommended flow is to create "a graph with links" and "a graph with links reattached to unlinked areas" (positive graph and negative graph), convolve the features of each node, and use the inner product to obtain a pair-wise score. Below is an explanation of the above formula.

> link prediction requires us to compute representation of pairs of nodes.
> DGL recommends you to treat the pairs of nodes as another graph, since you can describe a pair of nodes with an edge. In link prediction, you will have a positive graph consisting of all the positive examples as edges, and a negative graph consisting of all the negative examples. The positive graph and the negative graph will contain the same set of nodes as the original graph. This makes it easier to pass node features among multiple graphs for computation. As you will see later, you can directly feed the node representations computed on the entire graph to the positive and the negative graphs for computing pair-wise scores.

In [ ]:
# split train/test edge
u, v = g.edges()
u = u.detach().cpu().numpy()
v = v.detach().cpu().numpy()

eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.1)
train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

# linkが張られているnodeのペアと、張られていないnodeのペアに分ける
# Separate node pairs that are linked and node pairs that are not linked.
adj = sp.coo_matrix((np.ones(len(u)), (u, v)))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

In [ ]:
train_g = dgl.remove_edges(g, eids[:test_size]).to('cuda')

## Model

- This example uses a convolutional layer called SAGEConv
- The role of SAGEConv is almost the same as that of GraphConv in the previous section. It works even if you replace it. (I actually replaced them, but GraphConv was more accurate than SAGEConv as far as I tested.)
- https://docs.dgl.ai/en/0.8.x/generated/dgl.nn.pytorch.conv.SAGEConv.html

In [ ]:
from dgl.nn import SAGEConv

# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')
        #self.conv1 = GraphConv(in_feats, h_feats, allow_zero_in_degree = True) #It works
        #self.conv2 = GraphConv(h_feats, h_feats, allow_zero_in_degree = True) #It works

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h # nodeの特徴

- With predictor, the convolved per-node features are processed into per-link values (by taking the inner product of each node).

In [ ]:
import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            # apply_edges : (do the same operation on each edge of the graph)
            # Take the inner product of the source and destination node features (in this case 'h') and name it score (to be assigned to the edge feature)
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

In [ ]:
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

With the GraphSAGE class and DotPredictor (or MLPPredictor), we were able to extract the "inner product between nodes based on features of the entire GRAPH.  
From here, we need to define some correct/incorrect answer and the "error" from it, and then reduce it to improve the accuracy.

This error is defined by creating two types of graphs.  
**"graph using only the node that is actually linked" and "graph** with the exact opposite link tension (**).  
A graph that links to nodes that do not have links and excludes links to nodes that originally had actual edges.

After obtaining the above two types of **"graphs consisting of nodes that are always linked" and "graphs consisting of nodes that are always unlinked "**, the loss is minimized as follows.  
Then, minimize the loss as follows: 2.
(2) Predict the probability of the existence of the link in the combination of nodes that are actually linked → Take a predicted value between 0 and 1.  
   Ideally, we would like all the links to be 1 (with edges), so the loss is the difference between the ideal and the predicted value. 3.
3) Prediction of the probability of the existence of the link in the actual node combination with the link → Predicted value of 0 to 1 is taken.  
   Ideally, we want all of them to be "0 (no edges)" → The difference from the ideal is loss. 4.  
   
4. minimize the total loss of 2 and 3

In [ ]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes()).to('cuda')
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes()).to('cuda')

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes()).to('cuda')
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes()).to('cuda')

MLP Predictor (64)

In [ ]:
model = GraphSAGE(train_g.ndata['feat'].shape[1], 16).to('cuda')

# You can replace DotPredictor with MLPPredictor.

pred = MLPPredictor(64)
# pred = DotPredictor()

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).to('cuda')
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).detach().cpu().numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).detach().cpu().numpy()
    return roc_auc_score(labels, scores)

In [ ]:
# ----------- set up loss and optimizer -------------- #
# in this case, loss will in training loop
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

# ----------- training -------------------------------- #
all_logits = []
for e in range(100):
    # forward
    # まずはグラフ全体のnode特徴を求める
    h = model(train_g, train_g.ndata['feat'])
    
    # 正のグラフのリンク毎にリンクの有無を予測（正解は必ずリンクはある＝1）
    # pred関数は、node間の特徴の内積を出す（＝リンク単位の予測になる）
    # Predict whether or not a link exists for each link in a positive graph (correct answer is always a link = 1)
    # pred function produces the inner product of the features between nodes (= it is a link-by-link prediction)
    pos_score = pred(train_pos_g, h)
    
    # 負のグラフからリンクの有無を予測（実際の正解は必ずリンクはない＝0）
    # pred関数は、node間の特徴の内積を出す（＝リンク単位の予測になる）
    # Predict whether or not there is a link from a negative graph (actual correct answer is always no link = 0)
    # pred function produces an inner product of features between nodes (= it is a link-by-link prediction)
    neg_score = pred(train_neg_g, h)
    
    # pos_scoreとneg_scoreを結合した上で二値分類の精度を計算。誤差が最小になるようにnodeの特徴を更新していく
    # Calculate the accuracy of binary classification after combining pos_score and neg_score. Update node features to minimize error.
    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

# ----------- check results ------------------------ #
from sklearn.metrics import roc_auc_score
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_addmm)

# example3: graph classfication using GCN
https://docs.dgl.ai/tutorials/blitz/5_graph_classification.html

## load data

- This task requires many graph structures and labels for each graph, so it uses different data than cora's data.
- The sample data from dgl has graph data for protein composition, so we will use that data.

In [ ]:
import dgl.data

# Generate a synthetic dataset with 10000 graphs, ranging from 10 to 500 nodes.
dataset = dgl.data.GINDataset('PROTEINS', self_loop=True)

In [ ]:
print('graph num:', len(dataset))
print('Number of graph categories:', dataset.gclasses)

In [ ]:
# sample graph
i = 0
print(dataset[i])
print('target:', dataset[i][1])

nx_graph = dgl.to_networkx(dataset[i][0])
nx.draw_networkx(nx_graph)
plt.show()

i = 800
print(dataset[i])
print('target:', dataset[i][1])

nx_graph = dgl.to_networkx(dataset[i][0])
nx.draw_networkx(nx_graph)
plt.show()

## mini batch

Create mini-batch of graph data

In [ ]:
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler

num_examples = len(dataset)
num_train = int(num_examples * 0.8)

train_sampler = SubsetRandomSampler(torch.arange(num_train))
test_sampler = SubsetRandomSampler(torch.arange(num_train, num_examples))

train_dataloader = GraphDataLoader(
    dataset, sampler=train_sampler, batch_size=5, drop_last=False)
test_dataloader = GraphDataLoader(
    dataset, sampler=test_sampler, batch_size=5, drop_last=False)

In [ ]:
it = iter(train_dataloader)
batch = next(it)
print(batch)

In [ ]:
batched_graph, labels = batch
print('Number of nodes for each graph element in the batch:', batched_graph.batch_num_nodes())
print('Number of edges for each graph element in the batch:', batched_graph.batch_num_edges())

# Recover the original graph elements from the minibatch
graphs = dgl.unbatch(batched_graph)
print('The original graphs in the minibatch:')
print(graphs)

## model

- Convolution with graphconv
- The features (h) for each node are calculated, and the average of the features for each node is taken in the last "dgl.mean_nodes".

In [ ]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        g.ndata['h'] = h
        return dgl.mean_nodes(g, 'h') # Average the features obtained for each node, mean pooling

## train

In [ ]:
# Create the model with given dimensions
model = GCN(dataset.dim_nfeats, 16, dataset.gclasses)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in tqdm(range(50)):
    for batched_graph, labels in train_dataloader:
        pred = model(batched_graph, batched_graph.ndata['attr'].float())
        loss = F.cross_entropy(pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

num_correct = 0
num_tests = 0
for batched_graph, labels in test_dataloader:
    pred = model(batched_graph, batched_graph.ndata['attr'].float())
    num_correct += (pred.argmax(1) == labels).sum().item()
    num_tests += len(labels)

print('Test accuracy:', num_correct / num_tests)

# Sending the message within graph:  Heterogeneous Graph

https://docs.dgl.ai/guide/graph-heterogeneous.html

https://docs.dgl.ai/guide/training.html#guide-training-heterogeneous-graph-example

---

- Until now, we have only dealt with graphs with one type of information indicated by a node (e.g., node indicates "thesis", node indicates "protein constituent").
    - Such a graph is called a "homogeneous graph.
- However, we can also consider graphs with more than one type of node, such as "user→item" in marketing. Such a graph is called a "Heterogeneous Graph.
- There is a sample of creating a Heterogeneous Graph at https://docs.dgl.ai/guide/training.html#guide-training-heterogeneous-graph-example, so let's take a look!

The following graph is created with these elements. (node type/edge type/node type).  
('user', 'follow', 'user')
('user', 'followed-by', 'user')
('user', 'click', 'item')
('item', 'clicked-by', 'user')
('user', 'dislike', 'item')
('item', 'disliked-by', 'user')

In [ ]:
n_users = 1000
n_items = 500
n_follows = 3000
n_clicks = 5000
n_dislikes = 500
n_hetero_features = 10
n_user_classes = 5
n_max_clicks = 10

follow_src = np.random.randint(0, n_users, n_follows)
follow_dst = np.random.randint(0, n_users, n_follows)
click_src = np.random.randint(0, n_users, n_clicks)
click_dst = np.random.randint(0, n_items, n_clicks)
dislike_src = np.random.randint(0, n_users, n_dislikes)
dislike_dst = np.random.randint(0, n_items, n_dislikes)

hetero_graph = dgl.heterograph({
    ('user', 'follow', 'user'): (follow_src, follow_dst),
    ('user', 'followed-by', 'user'): (follow_dst, follow_src),
    ('user', 'click', 'item'): (click_src, click_dst),
    ('item', 'clicked-by', 'user'): (click_dst, click_src),
    ('user', 'dislike', 'item'): (dislike_src, dislike_dst),
    ('item', 'disliked-by', 'user'): (dislike_dst, dislike_src)})

hetero_graph.nodes['user'].data['feature'] = torch.randn(n_users, n_hetero_features)
hetero_graph.nodes['item'].data['feature'] = torch.randn(n_items, n_hetero_features)
hetero_graph.nodes['user'].data['label'] = torch.randint(0, n_user_classes, (n_users,))
hetero_graph.edges['click'].data['label'] = torch.randint(1, n_max_clicks, (n_clicks,)).float()
# randomly generate training masks on user nodes and click edges
hetero_graph.nodes['user'].data['train_mask'] = torch.zeros(n_users, dtype=torch.bool).bernoulli(0.6)
hetero_graph.edges['click'].data['train_mask'] = torch.zeros(n_clicks, dtype=torch.bool).bernoulli(0.6)

In [ ]:
hetero_graph

In [ ]:

# # Quirky hetero_graph cannot be directly visualized in networkX as it is. Too bad.
'''
nx_graph = dgl.to_networkx(hetero_graph)
nx.draw_networkx(nx_graph)
plt.show()

# DGLError: dgl.to_networkx only supports homogeneous graphs.
'''

- https://docs.dgl.ai/guide/training-node.html

---

Let's do node classfication on this graph.  
Suppose we have 5 types of user nodes and we want to predict which node each user belongs to.  
We use "dglnn.HeteroGraphConv" for convolution, summing the features for each type of edge, and further convolving the information.
- https://docs.dgl.ai/guide/training-node.html

In [ ]:
# Define a Heterograph Conv model

class RGCN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, rel_names):
        super().__init__()
        
        self.conv1 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(in_feats, hid_feats)
            for rel in rel_names}, aggregate='sum')
        self.conv2 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(hid_feats, out_feats)
            for rel in rel_names}, aggregate='sum')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        return h

In [ ]:
hetero_graph.etypes # edge type name

In [ ]:
# model
model = RGCN(n_hetero_features, 20, n_user_classes, hetero_graph.etypes)
user_feats = hetero_graph.nodes['user'].data['feature']
item_feats = hetero_graph.nodes['item'].data['feature']
labels = hetero_graph.nodes['user'].data['label']
train_mask = hetero_graph.nodes['user'].data['train_mask']

In [ ]:
# Features are for each type of node.
node_features = {'user': user_feats, 'item': item_feats}
h_dict = model(hetero_graph, {'user': user_feats, 'item': item_feats}) # 出力はuser/itemノードそれぞれ畳み込まれて出てくる。The output is collapsed for each user/item node.
h_user = h_dict['user']
h_item = h_dict['item']

In [ ]:
h_user # Predicted value of which label the user belongs to

In [ ]:
# train (20epochs)
opt = torch.optim.Adam(model.parameters())

for epoch in range(20):
    model.train()
    # forward propagation by using all nodes and extracting the user embeddings
    logits = model(hetero_graph, node_features)['user']
    # compute loss
    loss = F.cross_entropy(logits[train_mask], labels[train_mask])
    # Compute validation accuracy.  Omitted in this example.
    # backward propagation
    opt.zero_grad()
    loss.backward()
    opt.step()
    print(loss.item())

    # Save model if necessary.  Omitted in the example.

- Even with a wide variety of node and edge types, we have found that features can be learned, for example, with the HeteroGraphConv module!

Many Thanks,
Samane